# Handling Errors and Status Codes

This lab dives into how to **handle errors and status codes** in the `requests` library. Status codes communicate the outcome of an API request (e.g. **200** = success, **404** = not found, **500** = server error).

We'll focus on:
- Checking the **status code** and using the response object
- Raising errors automatically and what's available on error objects
- When to use the **exact status code** vs the **`response.ok`** boolean

## Quick recap: HTTP status codes

Status codes are **grouped by their first digit**:

| Range | Meaning |
|-------|--------|
| **1xx** | Informational |
| **2xx** | Success |
| **3xx** | Redirection |
| **4xx** | Client errors (something wrong on the requester's side) |
| **5xx** | Server errors (something wrong on the server's side) |

**Examples:**
- **200 OK** – success for GET requests
- **201 Created** – often returned for successful POST requests
- **301 Moved Permanently** – resource has been moved
- **404 Not Found** – resource does not exist
- **500 Internal Server Error** – server-side failure

Knowing these categories helps you **decide how to handle** each response. For example, if you implement retries, you would typically **retry only 5xx** (server might recover), not 4xx (the client request is wrong and retrying usually won't help). We'll come back to timeouts and retries later.

## 1. Using `response.status_code`

**`response.status_code`** gives you the **exact** HTTP code returned (e.g. `200`, `404`). You can compare it directly with numeric values when you need **fine-grained control** (e.g. different logic for 404 vs 403).

Below we use two endpoints from the GitHub API:
- **OK** – `/zen` returns a random sentence from the Zen of Python
- **Not found** – a non-existent path returns 404

In [1]:
import requests

GITHUB_ENDPOINT = "https://api.github.com"

urls = {
    "ok": f"{GITHUB_ENDPOINT}/zen",
    "not_found": f"{GITHUB_ENDPOINT}/this/endpoint/does/not/exist",
}

for description, url in urls.items():
    response = requests.get(url, timeout=5)
    print(f"{description}: status {response.status_code}")

ok: status 200
not_found: status 404


As expected: **ok** gets **200**, **not_found** gets **404**. Using `status_code` is ideal when you need to branch on the exact value (e.g. `if response.status_code == 404: ...`).

## 2. Using `response.ok` for a simple success check

If you only care whether the request **succeeded or not** (and not the exact code), use the **`response.ok`** attribute:

- **`True`** for status codes **below 400** (1xx, 2xx, 3xx)
- **`False`** for **4xx** and **5xx**

This gives you a quick boolean check without dealing with numeric codes.

In [2]:
for description, url in urls.items():
    response = requests.get(url, timeout=5)
    print(f"{description}: response.ok = {response.ok}")

ok: response.ok = True
not_found: response.ok = False


The **ok** endpoint yields `True`, **not_found** yields `False`. No need to remember that 200 is success—just check `response.ok`.

## 3. Combining `response.ok` with status code for clearer output

We can use **`response.ok`** in an **if/else** to branch on success, and still use **`response.status_code`** when we need to report *why* it failed.

In [3]:
for description, url in urls.items():
    response = requests.get(url, timeout=5)
    if response.ok:
        print(f"{description}: yes")
    else:
        print(f"{description}: no — failed with status {response.status_code}")

ok: yes
not_found: no — failed with status 404


This gives simple **yes/no** for success, plus the **status code** when the request failed, so you can log or handle errors more informatively.

**Summary:**
- Use **`response.status_code`** when you need the exact code (custom logic, retries, logging).
- Use **`response.ok`** when you only need to know "did it succeed?" and then add `status_code` in the failure branch if needed.

## 4. Automatic error handling with `raise_for_status()`

We can **automatically raise an exception** by calling **`response.raise_for_status()`**:
- It does **nothing** on 1xx, 2xx, and 3xx response codes.
- It **raises** `requests.exceptions.HTTPError` on **4xx** and **5xx**.

This fits the **"Easier to Ask Forgiveness than Permission" (EAFP)** style: write the **happy path** inside a `try` block, and handle exceptions in `except` clauses. The logic stays linear and often easier to read than nesting `if response.ok: ... else: ...`.

The raised **error object** has a **`response`** attribute—the original `Response`. You can inspect **`error.response.headers`** and **`error.response.text`** (or **`error.response.json()`** if the body is JSON) for valuable debugging information.

In [4]:
import requests
import json

for url in urls.values():
    print(f"Requesting {url}")
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        print("success")
    except requests.exceptions.HTTPError as error:
        print(f"HTTP error: {error} — status {error.response.status_code}")
        try:
            details = error.response.json()
            print("Error details:")
            print(json.dumps(details, indent=2))
        except ValueError:
            print(f"Non-JSON response body: {error.response.text[:100]}")

Requesting https://api.github.com/zen
success
Requesting https://api.github.com/this/endpoint/does/not/exist
HTTP error: 404 Client Error: Not Found for url: https://api.github.com/this/endpoint/does/not/exist — status 404
Error details:
{
  "message": "Not Found",
  "documentation_url": "https://docs.github.com/rest",
  "status": "404"
}


For the **ok** URL we get "success". For the **not_found** URL we get an HTTP error with status 404 and the error details from the response body (GitHub often returns JSON with a `message` field). If the body weren't JSON, we'd fall back to printing the first 100 characters of **`error.response.text`**.

## Common pitfalls and how to avoid them

| Pitfall | Why it matters | What to do |
|--------|----------------|------------|
| **Not checking errors** | Treating any response as successful can **mask failures** and let bugs creep in (e.g. bad data in your database). | Always use **`response.ok`**, **`raise_for_status()`**, or an explicit **status code** check. |
| **Catching too broadly** | Using a bare **`except Exception`** can **hide** HTTP errors and make debugging harder. You might want to handle **HTTP errors specifically** (e.g. retry logic for 5xx). | Catch **`requests.exceptions.HTTPError`** (and other specific exceptions) and handle them; let unexpected errors propagate. |
| **Ignoring the error body** | APIs often return **JSON or plain-text error messages** in the response body. That information is valuable for troubleshooting. | Even when an exception is raised, inspect **`error.response.text`** or **`error.response.json()`** to log or display the message the server sent. |